# Lecture 6 – γ p

**Invariant masses of particle pairs and Dalitz plots for another reaction with three particles in the final state $\gamma p\rightarrow \pi^+\pi^-p$.**

Prepare the notebook with the preambles for the inclusion of pandas, numpy and matplotlib.pyplot:

In [ ]:
%pip install -q gdown matplotlib numpy pandas pylorentz scipy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import gdown

output_path = gdown.cached_download(
    url="https://drive.google.com/uc?id=1qiYjPbR5nx3_Sw7MXuUKhNAUpkXPoxYh",
    path="data/lecture6-gammap-data.csv",
    md5="38cf5bf915318df756a21a82ad9e4afa",
    quiet=True,
)
data = pd.read_csv(output_path)

Inspect the data file and format. The file contains the 4-momenta of the particles of the reaction $\gamma p \rightarrow \pi^+\pi^- p$, the last column corresponds to the 3rd component of the photon momentum (up to 2.5 GeV/c), which travels along the $z$ axis. Since the target was made of HD, to select the data interacting on protons only a cut on the missing momentum of the reaction was made.

In [ ]:
data.head()

The columns headers present some trailing blanks, that must be dropped to be able to use correctly the DataFormat structure (if not, they deliver an error message). To do so, the *str.strip()* function must be used beforehand to reformat the column shape.
In the following commands in the cell, columns are shown, overall with the data.columns command, and per single variable (like *data.p2x*). If the format is correct, no error should appear.

In [ ]:
data.columns = data.columns.str.strip()
data.p2x

Evaluate the invariant masses (squared and linear) for particle pairs, in a similar way as done in the first example.

Let's plot the evaluated invariant masses. First, though, let's plot the antineutron momentum to see how the distribution looks like.

Let's plot now the invariant masses distributions for the three pion pairs:

2D distributions: Dalitz plots

How do Dalitz plots look like with MonteCarlo generated data?
Repeat the previous procedures with a new file, corresponding to generated data from the same reaction, and compare the shapes (statistics are different)

In [ ]:
path_mc = gdown.cached_download(
    url="https://drive.google.com/uc?id=11J0xaQLRMxzgQLXEhXZb_u4mnxp8RVPO",
    path="data/lecture6-gammap-mc.csv",
    md5="04152c69c802b13a55a3ffb3d07012d1",
    quiet=True,
)
mc = pd.read_csv(path_mc)
mc.columns = mc.columns.str.strip()

The phase space simulation was done extracting the photon momentum from the real data distribution.

Let's see the invariant masses histogram shapes using phase space Monte Carlo events.

And now the Dalitz plots:



---



Now, let's take a look to the missing mass distribution (final - initial
state). Let's start from the real data using the pylorentz package to build 4-vectors (first, work out *relativisticKinematics.ipynb* notebook).

In [ ]:
from pylorentz import Momentum4

# final state
p1T = np.sqrt(data.p1x**2 + data.p1y**2)
p1mod = np.sqrt(p1T**2 + data.p1z**2)
eta1 = np.arcsinh(data.p1z / p1T)
phi1 = np.arctan2(data.p1y, data.p1x)
pionPlus = Momentum4.e_eta_phi_p(data.E1, eta1, phi1, p1mod)

p2T = np.sqrt(data.p2x**2 + data.p2y**2)
p2mod = np.sqrt(p2T**2 + data.p2z**2)
eta2 = np.arcsinh(data.p2z / p2T)
phi2 = np.arctan2(data.p2y, data.p2x)
pionMinus = Momentum4.e_eta_phi_p(data.E2, eta2, phi2, p2mod)

p3T = np.sqrt(data.p3x**2 + data.p3y**2)
p3mod = np.sqrt(p3T**2 + data.p3z**2)
eta3 = np.arcsinh(data.p3z / p3T)
phi3 = np.arctan2(data.p3y, data.p3x)
proton = Momentum4.e_eta_phi_p(data.E3, eta3, phi3, p3mod)

# initial state
# len is the number of events read by the csv file
len = len(data.pgamma)
pgamT = np.zeros(len)
pgammod = data.pgamma
etagam = 1.0e11 * np.ones(len)
phigam = np.zeros(len)
massGamma = 0.0
Egamma = np.sqrt(massGamma**2 + pgammod**2)
photon = Momentum4.e_eta_phi_p(Egamma, etagam, phigam, pgammod)

# the target (proton) is at rest
massProton = 0.93827
ETgt = massProton * np.ones(len)
etaTgt = np.zeros(len)
phiTgt = np.zeros(len)
pTgt = np.zeros(len)
protonTarget = Momentum4.e_eta_phi_p(ETgt, etaTgt, phiTgt, pTgt)

In [ ]:
init = photon + protonTarget
final = pionPlus + pionMinus + proton
missingMomentum = final - init

plt.hist(missingMomentum.m2, bins=200, color="palegreen", range=(-0.02, 0.01))
plt.xlabel(R"Missing mass squared [$(\mathrm{GeV}/c^2)^2$]")
plt.ylabel(R"Entries/($5x10^{-5}\;(\mathrm{GeV}/c^2)^2$)")
plt.title("Missing momentum \n")
plt.show()

The missing mass square is always negative: this means that the total energy of the initial state exceeds the measured energy of the final state, so there is (likely) a missing particle which carries away some energy. We know that the reaction occurred on a HD molecule as a target: this means that a recoiling neutron is present in all cases when the reaction occurred on a deuteron nucleus: $\gamma p(n)\rightarrow \pi^+\pi^- p (n)$. In this case, moreover, the hit proton is not at rest but it may have a momentum (called Fermi momentum) which, in the deuteron center-of-mass, is roughly distributed as a gaussian 50 MeV/c/ wide, with maximum at abouth 270 MeV/c. The missing mass momentum distribution shows the effect of the presence of a non-null Fermi momentum, and the possible contribution to the reaction kinematics of the spectator neutron.

In the data selection procedure, the Fermi momentum was required not to exceed 100 MeV/c to preserve the condition that the neutron is a spectator in the reaction occurring on deuteron. Let's see the shape of the missing momentum distribution:  

In [ ]:
plt.hist(missingMomentum.p, bins=150, color="mediumspringgreen", range=(0.0, 0.150))
plt.xlabel("Missing momentum [GeV/c]")
plt.ylabel("Entries/(10 MeV/c)")
plt.title("Missing momentum \n")
plt.show()

Let's now consider the missing mass recoiling against the neutral dipion: in an exclusive reaction we expect it to peak at the proton mass. Is the PID selection of our sample acceptable?

In [ ]:
dipion = pionPlus + pionMinus
missingMomentumDipion = init - dipion

plt.hist(missingMomentumDipion.m2, bins=200, color="violet", range=(0.0, 2))
plt.xlabel(R"Missing mass squared [$(\mathrm{GeV}/c^2)^2$]")
plt.ylabel(R"Entries/($0.01\;(\mathrm{GeV}/c^2)^2$)")
plt.title("Missing momentum recoiling against the dipion\n")
plt.show()

Let's visualize the scatter plot of the two missing masses squared:

In [ ]:
fig = plt.figure()
plt.hist2d(
    missingMomentumDipion.m2,
    missingMomentum.m2,
    bins=200,
    range=[[0.80, 1.0], [-0.003, 0.001]],
    cmap="rainbow",
)
plt.xlabel(R"Dipion M.M. squared [$(\mathrm{GeV}/c^2)^2$]")
plt.ylabel(R"Reaction M.M. squared [$(\mathrm{GeV}/c^2)^2$)")
plt.show()

---

## Let's do some fits
Which is the maximum of the missing momentum, and where is the peak of the
missing mass distribution?
Let's attempt two fits with a single gaussian function.


In [ ]:
from scipy.optimize import curve_fit

# normalize the histogram to 1
lowEdge = 0.015
upEdge = 0.035
hist, bin_edges = np.histogram(missingMomentum.p, 100, range=(lowEdge, upEdge))
integralHist = sum(hist)
hist = hist / integralHist

n = hist.size
x_hist = np.zeros((n), dtype=float)
for ii in range(n):
    x_hist[ii] = (bin_edges[ii + 1] + bin_edges[ii]) / 2

y_hist = hist

# Calculating the Gaussian PDF values given Gaussian parameters and random variable X


def gaus(X, C, X_mean, sigma):
    return C * np.exp(-((X - X_mean) ** 2) / (2 * sigma**2))


mean = sum(x_hist * y_hist) / sum(y_hist)
sigma = sum(y_hist * (x_hist - mean) ** 2) / sum(y_hist)

# Gaussian least-square fitting process
param_optimised, param_covariance_matrix = curve_fit(
    gaus, x_hist, y_hist, p0=[max(y_hist), mean, sigma], maxfev=5000
)

# print fit Gaussian parameters
print("Fit parameters: ")
print("=====================================================")
print("C = ", param_optimised[0], "+-", np.sqrt(param_covariance_matrix[0, 0]))
print("X_mean =", param_optimised[1], "+-", np.sqrt(param_covariance_matrix[1, 1]))
print("sigma = ", param_optimised[2], "+-", np.sqrt(param_covariance_matrix[2, 2]))
print("\n")


# STEP 4: PLOTTING THE GAUSSIAN CURVE -----------------------------------------
fig = plt.figure()
x_hist_2 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
# this plots the curve only
plt.plot(x_hist_2, gaus(x_hist_2, *param_optimised), color="red", label="Gaussian fit")
plt.legend()

# plot the experimental point of the portion of spectrum to be fitted
plt.scatter(x_hist, y_hist, color="mediumspringgreen")

# Normalise the histogram values
weights = np.ones_like(y_hist) / y_hist.size
# plt.hist(x_hist, weights=weights)
# plt.hist(x_hist)
# plt.hist(hist)


# setting the label,title and grid of the plot
plt.xlabel("Data: Random variable")
plt.ylabel("Probability")
plt.grid("on")
plt.show()

In [ ]:
# normalize the histogram to 1 and then to ratio of the maximum values
histfull, bin_edgesf = np.histogram(missingMomentum.p, 100, range=(0.0, 0.10))
n = histfull.size
xf_hist = np.zeros((n), dtype=float)
ilow = 0
iup = 0
# find the bin numbers corresponding to fitted histrogram edges
for ii in range(n):
    xf_hist[ii] = (bin_edgesf[ii + 1] + bin_edgesf[ii]) / 2
    # find the closest bin to edges
    if bin_edgesf[ii + 1] <= lowEdge:
        ilow = ii
    if bin_edgesf[ii + 1] <= upEdge:
        iup = ii


integralFull = sum(histfull)
histfull = histfull / integralFull
yf_hist = histfull
histfull = histfull * np.max(y_hist) / np.max(yf_hist)
yf_hist = histfull

plt.scatter(xf_hist, yf_hist, color="mediumspringgreen")

x_hist_3 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
plt.plot(x_hist_3, gaus(x_hist_3, *param_optimised), color="red")  # was 'r.:'
plt.show()

In [ ]:
# normalize the histogram to 1
lowEdge = -0.001
upEdge = 0.0
hist, bin_edges = np.histogram(missingMomentum.m2, 100, range=(lowEdge, upEdge))
integralHist = sum(hist)
hist = hist / integralHist

n = hist.size
x_hist = np.zeros((n), dtype=float)
for ii in range(n):
    x_hist[ii] = (bin_edges[ii + 1] + bin_edges[ii]) / 2

y_hist = hist
mean = sum(x_hist * y_hist) / sum(y_hist)
sigma = sum(y_hist * (x_hist - mean) ** 2) / sum(y_hist)
# Gaussian least-square fitting process
param_optimised, param_covariance_matrix = curve_fit(
    gaus, x_hist, y_hist, p0=[max(y_hist), mean, sigma], maxfev=5000
)

# print fit Gaussian parameters
print("Fit parameters: ")
print("=====================================================")
print("C = ", param_optimised[0], "+-", np.sqrt(param_covariance_matrix[0, 0]))
print("X_mean =", param_optimised[1], "+-", np.sqrt(param_covariance_matrix[1, 1]))
print("sigma = ", param_optimised[2], "+-", np.sqrt(param_covariance_matrix[2, 2]))
print("\n")

# save the values for later
mean1Fit = param_optimised[1]
sigma1Fit = param_optimised[2]

# STEP 4: PLOTTING THE GAUSSIAN CURVE -----------------------------------------
fig = plt.figure()
x_hist_2 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
# this plots the curve only
plt.plot(x_hist_2, gaus(x_hist_2, *param_optimised), color="red", label="Gaussian fit")
plt.legend()

# plot the experimental point of the portion of spectrum to be fitted
plt.scatter(x_hist, y_hist, color="palegreen")

# setting the label,title and grid of the plot
plt.xlabel("Data: Random variable")
plt.ylabel("Probability")
plt.grid("on")
plt.show()

In [ ]:
# normalize the histogram to 1 and then to ratio of the maximum values
histfull, bin_edgesf = np.histogram(missingMomentum.m2, 100, range=(-0.005, 0.005))
n = histfull.size
xf_hist = np.zeros((n), dtype=float)

# find the bin numbers
for ii in range(n):
    xf_hist[ii] = (bin_edgesf[ii + 1] + bin_edgesf[ii]) / 2

integralFull = sum(histfull)
histfull = histfull / integralFull
yf_hist = histfull
histfull = histfull * np.max(y_hist) / np.max(yf_hist)
yf_hist = histfull

plt.scatter(xf_hist, yf_hist, color="palegreen")
x_hist_3 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
plt.plot(x_hist_3, gaus(x_hist_3, *param_optimised), color="red")
plt.show()

The fit of the missing mass peak is not really very good, one should add some sort of background on the left hand side of the peak, say a 3rd degree polynomial

In [ ]:
# normalize the histogram to 1
lowEdge = -0.0007
upEdge = 0.0001
hist, bin_edges = np.histogram(missingMomentum.m2, 100, range=(lowEdge, upEdge))
integralHist = sum(hist)
hist = hist / integralHist

n = hist.size
x_hist = np.zeros((n), dtype=float)
for ii in range(n):
    x_hist[ii] = (bin_edges[ii + 1] + bin_edges[ii]) / 2

y_hist = hist
mean = sum(x_hist * y_hist) / sum(y_hist)
sigma = sum(y_hist * (x_hist - mean) ** 2) / sum(y_hist)
# mean = mean1Fit
# sigma = sigma1Fit
P1 = 0.0
P2 = 0.0
P3 = 0.0
P4 = 0.0

# Calculating the Gaussian PDF values given Gaussian parameters and random variable X


def gausAndBCK(X, C, X_mean, sigma, P1, P2, P3, P4):
    return C * np.exp(-((X - X_mean) ** 2) / (2 * sigma**2)) + (
        P1 + P2 * X + P3 * X * X + P4 * X * X * X
    )


def BCK(X, P1, P2, P3, P4):
    return P1 + P2 * X + P3 * X * X + P4 * X * X * X


# Gaussian+BCK least-square fitting process
param_optimised, param_covariance_matrix = curve_fit(
    gausAndBCK,
    x_hist,
    y_hist,
    p0=[max(y_hist), mean, sigma, P1, P2, P3, P4],
    maxfev=10000,
)

# print fit Gaussian parameters
print("Fit parameters: ")
print("=====================================================")
print("C = ", param_optimised[0], "+-", np.sqrt(param_covariance_matrix[0, 0]))
print("X_mean =", param_optimised[1], "+-", np.sqrt(param_covariance_matrix[1, 1]))
print("sigma = ", param_optimised[2], "+-", np.sqrt(param_covariance_matrix[2, 2]))
print("\n")
print("missing mass at the peak ", np.sqrt(abs(param_optimised[1])), " GeV/c2 \n")

param_optimised_gauss = []
param_optimised_bck = []
param_optimised_gauss.append(param_optimised[0])
param_optimised_gauss.append(param_optimised[1])
param_optimised_gauss.append(param_optimised[2])
param_optimised_bck.append(param_optimised[3])
param_optimised_bck.append(param_optimised[4])
param_optimised_bck.append(param_optimised[5])
param_optimised_bck.append(param_optimised[6])

fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(7, 4))
x_hist_2 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
# this plots the curve only
ax[0].plot(
    x_hist_2, gausAndBCK(x_hist_2, *param_optimised), color="red", label="Gaussian fit"
)
ax[0].plot(x_hist_2, gaus(x_hist_2, *param_optimised_gauss), color="cyan")
ax[0].plot(x_hist_2, BCK(x_hist_2, *param_optimised_bck), color="magenta")

# plot the experimental point of the portion of spectrum to be fitted
ax[0].scatter(x_hist, y_hist, color="palegreen")

# setting the label,title and grid of the plot

ax[0].set_xlabel("Data: Random variable")
ax[0].set_ylabel("Probability")

# full plot
histfull, bin_edgesf = np.histogram(missingMomentum.m2, 100, range=(-0.005, 0.005))
n = histfull.size
xf_hist = np.zeros((n), dtype=float)

# find the bin numbers
for ii in range(n):
    xf_hist[ii] = (bin_edgesf[ii + 1] + bin_edgesf[ii]) / 2

integralFull = sum(histfull)
histfull = histfull / integralFull
yf_hist = histfull
histfull = histfull * np.max(y_hist) / np.max(yf_hist)
yf_hist = histfull

plt.scatter(xf_hist, yf_hist, color="palegreen")
x_hist_3 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
plt.plot(x_hist_3, gausAndBCK(x_hist_3, *param_optimised), color="red")
plt.show()

It looks like the Gaussian contribution is refused by the fit.

Let's make a similar fit for the missing mass recoiling against the dipion:

In [ ]:
# normalize the histogram to 1
lowEdge = 0.8
upEdge = 1.0
hist, bin_edges = np.histogram(missingMomentumDipion.m2, 100, range=(lowEdge, upEdge))
integralHist = sum(hist)
hist = hist / integralHist

n = hist.size
x_hist = np.zeros((n), dtype=float)
for ii in range(n):
    x_hist[ii] = (bin_edges[ii + 1] + bin_edges[ii]) / 2

y_hist = hist
mean = sum(x_hist * y_hist) / sum(y_hist)
sigma = sum(y_hist * (x_hist - mean) ** 2) / sum(y_hist)
# mean = mean1Fit
# sigma = sigma1Fit
P1 = 0.0
P2 = 0.0
P3 = 0.0
P4 = 0.0

# Calculating the Gaussian PDF values given Gaussian parameters and random variable X


def gausAndBCK(X, C, X_mean, sigma, P1, P2, P3, P4):
    return C * np.exp(-((X - X_mean) ** 2) / (2 * sigma**2)) + (
        P1 + P2 * X + P3 * X * X + P4 * X * X * X
    )


def BCK(X, P1, P2, P3, P4):
    return P1 + P2 * X + P3 * X * X + P4 * X * X * X


# Gaussian+BCK least-square fitting process
param_optimised, param_covariance_matrix = curve_fit(
    gausAndBCK,
    x_hist,
    y_hist,
    p0=[max(y_hist), mean, sigma, P1, P2, P3, P4],
    maxfev=10000,
)

# print fit Gaussian parameters
print("Fit parameters: ")
print("=====================================================")
print("C = ", param_optimised[0], "+-", np.sqrt(param_covariance_matrix[0, 0]))
print("X_mean =", param_optimised[1], "+-", np.sqrt(param_covariance_matrix[1, 1]))
print("sigma = ", param_optimised[2], "+-", np.sqrt(param_covariance_matrix[2, 2]))
print("\n")
print("missing mass at the peak ", np.sqrt(abs(param_optimised[1])), " GeV/c2 \n")

param_optimised_gauss = []
param_optimised_bck = []
param_optimised_gauss.append(param_optimised[0])
param_optimised_gauss.append(param_optimised[1])
param_optimised_gauss.append(param_optimised[2])
param_optimised_bck.append(param_optimised[3])
param_optimised_bck.append(param_optimised[4])
param_optimised_bck.append(param_optimised[5])
param_optimised_bck.append(param_optimised[6])

fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(7, 4))
x_hist_2 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
# this plots the curve only
ax[0].plot(
    x_hist_2, gausAndBCK(x_hist_2, *param_optimised), color="red", label="Gaussian fit"
)
ax[0].plot(x_hist_2, gaus(x_hist_2, *param_optimised_gauss), color="cyan")
ax[0].plot(x_hist_2, BCK(x_hist_2, *param_optimised_bck), color="magenta")

# plot the experimental point of the portion of spectrum to be fitted
ax[0].scatter(x_hist, y_hist, color="violet")

# setting the label,title and grid of the plot

ax[0].set_xlabel("Missing mass recoiling against dipion")
ax[0].set_ylabel("Probability")

# full plot
histfull, bin_edgesf = np.histogram(missingMomentumDipion.m2, 100, range=(0.5, 1.2))
n = histfull.size
xf_hist = np.zeros((n), dtype=float)

# find the bin numbers
for ii in range(n):
    xf_hist[ii] = (bin_edgesf[ii + 1] + bin_edgesf[ii]) / 2

integralFull = sum(histfull)
histfull = histfull / integralFull
yf_hist = histfull
histfull = histfull * np.max(y_hist) / np.max(yf_hist)
yf_hist = histfull

plt.scatter(xf_hist, yf_hist, color="violet")
x_hist_3 = np.linspace(np.min(x_hist), np.max(x_hist), 100)
plt.plot(x_hist_3, gausAndBCK(x_hist_3, *param_optimised), color="red")
plt.plot(x_hist_3, gaus(x_hist_3, *param_optimised_gauss), color="cyan")
plt.plot(x_hist_3, BCK(x_hist_3, *param_optimised_bck), color="magenta")
plt.show()